In [67]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model

# Loading the Data

In [56]:
df = pd.read_csv('data.csv')

<ipython-input-56-6456521db808>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [57]:
df.head()

,fdc_id,brand_owner,brand_name,ingredients,not_a_significant_source_of,branded_food_category,is_gluten,contains_meat,contains_shellfish,contains_nuts,contains_diary,contains_soy,contains_egg,contains_animal_byproducts,is_vegan,is_vegetarian,is_pescatarian
0,1105904,Richardson Oilseed Products (US) Limited,NaN,Vegetable Oil,NaN,Oils Edible,False,False,False,False,False,False,False,False,True,True,True
1,1105905,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",NaN,Herbs/Spices/Extracts,False,True,False,False,False,False,False,False,False,False,False
2,1105906,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",NaN,Prepared Soups,False,True,True,False,True,True,False,True,False,False,False
3,1105907,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",NaN,Prepared Soups,False,False,False,False,True,True,False,True,False,True,True
4,1105908,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",NaN,Herbs/Spices/Extracts,False,True,False,False,False,False,False,False,False,False,False


In [58]:
df.ingredients[31]

'MADE FROM: ENRICHED WHEAT FLOUR (FLOUR, NIACIN, REDUCED IRON, THIAMINE MONONITRATE, RIBOFLAVIN, FOLIC ACID), SUGAR, BUTTER, HIGH FRUCTOSE CORN SYRUP, CORN SYRUP, CORNSTARCH, VEGETABLE OILS (PALM AND/OR SOYBEAN AND HYDROGENATED SOYBEAN), CONTAINS 2% OR LESS OF: APRICOT PUREE CONCENTRATE, RASPBERRY CONCENTRATE, EGGS, NONFAT MILK, PECTIN, SOY PROTEIN, BAKING SODA, CITRIC ACID, SALT, CREAM OF TARTAR, SODIUM CITRATE, CARAMEL COLOR.  CONTAINS: WHEAT, MILK, EGG, SOY.'

### Splitting the Data

In [59]:
from sklearn.model_selection import train_test_split

In [60]:
# Step 1: Split into train and temporary set (70-30)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)

# Step 2: Split temporary set into validation and test sets (50-50)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [51]:
train_df.shape

(137761, 17)

# Setting up the BERT baseline

In [13]:
from transformers import BertTokenizer, TFBertModel

from transformers import logging
logging.set_verbosity_error()

In [15]:
checkpoint = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(checkpoint)
bert_model = TFBertModel.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [17]:
MAX_SEQUENCE_LENGTH = 128

In [77]:
train_df['ingredients'] = train_df['ingredients'].fillna('', inplace=False)
test_df['ingredients'] = test_df['ingredients'].fillna('', inplace=False)
val_df['ingredients'] = val_df['ingredients'].fillna('', inplace=False)


In [79]:
train_df['ingredients'] = train_df['ingredients'].astype(str)
test_df['ingredients'] = test_df['ingredients'].astype(str)
val_df['ingredients'] = val_df['ingredients'].astype(str)


In [83]:
train_examples_str = train_df['ingredients'].tolist()
test_examples_str = test_df['ingredients'].tolist()
val_examples_str = val_df['ingredients'].tolist()


### Tokenizing the inputs

In [84]:
# tokenizing training set
bert_train_tokenized = bert_tokenizer(train_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

bert_train_inputs = [bert_train_tokenized.input_ids,
                     bert_train_tokenized.token_type_ids,
                     bert_train_tokenized.attention_mask]

In [85]:
bert_test_tokenized = bert_tokenizer(test_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

bert_test_inputs = [bert_test_tokenized.input_ids,
                     bert_test_tokenized.token_type_ids,
                     bert_test_tokenized.attention_mask]

In [86]:
bert_val_tokenized = bert_tokenizer(val_examples_str,
              max_length=MAX_SEQUENCE_LENGTH,
              truncation=True,
              padding='max_length',
              return_tensors='tf')

bert_val_inputs = [bert_val_tokenized.input_ids,
                     bert_val_tokenized.token_type_ids,
                     bert_val_tokenized.attention_mask]

## Baseline Model

In [87]:
train_labels = ['is_gluten', 'contains_shellfish', 'contains_nuts', 'contains_diary', 'contains_soy', 'is_vegan', 'is_vegetarian', 'is_pescatarian']

In [ ]:
# This is from the class notebook - I know we need to change this to a multiclassification model
# I'm not too sure if the one-hot encoding setup for the labels work here but I need to think on it

def create_bert_classification_model(bert_model,
                                     num_train_layers=0,
                                     hidden_size = 200,
                                     dropout=0.3,
                                     learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes
    """

    input_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)

    #pooler_token = bert_out[1]
    cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooler_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)


    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])

    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                                 metrics='accuracy')

    return classification_model